In [ ]:
import os,shutil
import h5py
from h5glance import H5Glance
import matplotlib.pyplot as plt

# Helpers
from SimEx.Utilities.Units import meter, electronvolt, joule, radian

# PMI
from SimEx.Calculators.XMDYNDemoPhotonMatterInteractor import XMDYNDemoPhotonMatterInteractor

# Simple Beam Parameters
from SimEx.Parameters.PhotonBeamParameters import PhotonBeamParameters

# Diffraction
from SimEx.Parameters.DetectorGeometry import DetectorGeometry, DetectorPanel
from SimEx.Parameters.SingFELPhotonDiffractorParameters import SingFELPhotonDiffractorParameters
from SimEx.Calculators.SingFELPhotonDiffractor import SingFELPhotonDiffractor

# Analysis
from SimEx.Analysis.XFELPhotonAnalysis import XFELPhotonAnalysis
from SimEx.Analysis.DiffractionAnalysis import DiffractionAnalysis

import matplotlib.colors as colors

## Data path setup

In [ ]:
data_path = '../../../data/simulation/'

## Beam

In [ ]:
beam = PhotonBeamParameters(
            photon_energy=18.86e3 * electronvolt,
            beam_diameter_fwhm=1.0e-6 * meter,
            pulse_energy=1.0e-3 * joule,
        )

## Detector Setting

In [ ]:
p0 = DetectorPanel(
            ranges={
                'fast_scan_min': 0,
                'fast_scan_max': 100,
                'slow_scan_min': 0,
                'slow_scan_max': 200
            },
            pixel_size=220e-6 * meter,
            photon_response=1.0,
            distance_from_interaction_plane=0.13 * meter,
            corners={
                'x': -50,
                'y': 1
            },
            fast_scan_xyz = '1.0x',
            slow_scan_xyz = '1.0y'
        )
p1 = DetectorPanel(
            ranges={
                'fast_scan_min': 0,
                'fast_scan_max': 100,
                'slow_scan_min': 0,
                'slow_scan_max': 200
            },
            pixel_size=220e-6 * meter,
            photon_response=1.0,
            distance_from_interaction_plane=0.13 * meter,
            corners={
                'x': -50,
                'y': -201
            },
            fast_scan_xyz = '1.0x',
            slow_scan_xyz = '1.0y'
        )
detector_geometry = DetectorGeometry(panels=[p0,p1])

In [ ]:
detector_geometry.serialize()

## Clean up previous diffraction run

In [ ]:
dirs_to_remove=['diffr']
files_to_remove=['diffr.h5']

for d in dirs_to_remove:
    if os.path.isdir(d):
        shutil.rmtree(d)
for f in files_to_remove:
    if os.path.isfile(f):
        os.remove(f)

## Diffractor Settings

In [ ]:
diffraction_parameters = SingFELPhotonDiffractorParameters(
                                               uniform_rotation=False,
                                               slice_interval=1,
                                               number_of_slices=1,
                                               number_of_diffraction_patterns=2,
                                               detector_geometry=detector_geometry,
                                               beam_parameters = beam,
                                               forced_mpi_command='mpirun -np 1',
                                              )

In [ ]:
diffractor = SingFELPhotonDiffractor(parameters=diffraction_parameters,
                                     input_path='3WUL.pdb',
                                     output_path="diffr")

In [ ]:
diffractor._run()
diffractor.saveH5()

In [ ]:
H5Glance('diffr/diffr_out_0000001.h5')

In [ ]:
H5Glance('diffr.h5')

In [ ]:
with h5py.File('diffr.h5','r') as f:
    for i,iteration in enumerate(f['data']):
        if i < 1:
            diffr = f['data'][iteration]['diffr'][0]
            plt.figure()
            plt.imshow(diffr,norm=colors.LogNorm(vmin=diffr.min(), vmax=diffr.max()),origin='lower')
            diffr = f['data'][iteration]['diffr'][1]
            plt.figure()
            plt.imshow(diffr,norm=colors.LogNorm(vmin=diffr.min(), vmax=diffr.max()),origin='lower')